In [1]:
import datetime

import pandas as pd

In [32]:
df_general = pd.read_csv("./datasets/reference_data/general_table.csv")
#keep only >=18 years old
df_general = df_general[df_general["age"] >= 18]
df_general["admissiontime"] = pd.to_datetime(df_general["admissiontime"])
df_general.reset_index(drop=True, inplace=True)
df_general.describe(include="all")

ipykernel_launcher:6: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.


,patientid,admissiontime,sex,age,discharge_status
count,33905.000000,33905,33905,33905.000000,33666
unique,NaN,33804,2,NaN,2
top,NaN,2118-10-02 01:00:00,M,NaN,alive
freq,NaN,2,21767,NaN,31604
first,NaN,2102-11-27 06:45:00,NaN,NaN,NaN
last,NaN,2198-09-03 16:35:00,NaN,NaN,NaN
mean,16953.000000,NaN,NaN,63.523964,NaN
std,9787.674775,NaN,NaN,15.196436,NaN
min,1.000000,NaN,NaN,20.000000,NaN
25%,8477.000000,NaN,NaN,55.000000,NaN


In [33]:
patients_more_24h = list()
df_parts = pd.read_csv("./datasets/raw_stage/observation_tables/observation_tables_index.csv")
df_parts.sort_values(by=["part"], inplace=True)
for part in df_parts["part"].unique():
    print(part)
    df_part_X = pd.read_parquet(f"./datasets/raw_stage/observation_tables/parquet/part-{part}.parquet")
    for patient in df_parts[df_parts["part"] == part]["patientid"]:
        df_part_X_patient_Y = df_part_X[df_part_X["patientid"] == patient].copy()
        df_part_X_patient_Y.sort_values(by=["datetime", "entertime"], inplace=True)
        #keep only >=1 day stay
        #old behaviour but not all patients have the first datetime equal to the admissiontime
        #if (df_part_X_patient_Y.iloc[-1]["datetime"] - df_part_X_patient_Y.iloc[0]["datetime"]) >= datetime.timedelta(days=1):
        if (df_part_X_patient_Y.iloc[-1]["datetime"] - df_general[df_general["patientid"] == patient].iloc[0]["admissiontime"]) >= datetime.timedelta(days=1):
            patients_more_24h.append(patient)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249


In [34]:
print(len(patients_more_24h), len(df_general))

16787 33905


In [35]:
lactate_var_ids = [24000524, 24000732, 24000485]
patients_more_24h_2lactate = list()
df_parts = pd.read_csv("./datasets/raw_stage/observation_tables/observation_tables_index.csv")
df_parts.sort_values(by=["part"], inplace=True)
for part in df_parts["part"].unique():
    print(part)
    df_part_X = pd.read_parquet(f"./datasets/raw_stage/observation_tables/parquet/part-{part}.parquet")
    for patient in df_parts[(df_parts["part"] == part) & (df_parts["patientid"].isin(patients_more_24h))]["patientid"]:
        df_part_X_patient_Y = df_part_X[df_part_X["patientid"] == patient].copy()
        #keep only >=2 lactate measurements
        if len(df_part_X_patient_Y[df_part_X_patient_Y["variableid"].isin(lactate_var_ids)]) >= 2:
            patients_more_24h_2lactate.append(patient)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249


In [36]:
print(len(patients_more_24h_2lactate), len(patients_more_24h), len(df_general))

15209 16787 33905


In [37]:
lactate_var_ids = [24000524, 24000732, 24000485]
patients_more_24h_2lactate_in_first_48h = list()
df_parts = pd.read_csv("./datasets/raw_stage/observation_tables/observation_tables_index.csv")
df_parts.sort_values(by=["part"], inplace=True)
for part in df_parts["part"].unique():
    print(part)
    df_part_X = pd.read_parquet(f"./datasets/raw_stage/observation_tables/parquet/part-{part}.parquet")
    for patient in df_parts[(df_parts["part"] == part) & (df_parts["patientid"].isin(patients_more_24h_2lactate))]["patientid"]:
        df_part_X_patient_Y = df_part_X[df_part_X["patientid"] == patient].copy()
        df_part_X_patient_Y.sort_values(by=["datetime", "entertime"], inplace=True)
        post_2d = df_part_X_patient_Y.iloc[0]["datetime"] + datetime.timedelta(days=2)
        #keep only first 48h
        df_part_X_patient_Y = df_part_X_patient_Y[df_part_X_patient_Y["datetime"] <= post_2d]
        #keep only >=2 lactate measurements in the first 48h
        if len(df_part_X_patient_Y[df_part_X_patient_Y["variableid"].isin(lactate_var_ids)]) >= 2:
            patients_more_24h_2lactate_in_first_48h.append(patient)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249


In [38]:
print(len(patients_more_24h_2lactate_in_first_48h), len(patients_more_24h_2lactate), len(patients_more_24h), len(df_general))

15048 15209 16787 33905


In [39]:
with open("./results/patients_18+_morethan_1d.txt", "w") as f:
    for x in patients_more_24h:
        f.write(f"{x}\n")
with open("./results/patients_18+_morethan_1d_morethan_2_lactate_measurements.txt", "w") as f:
    for x in patients_more_24h_2lactate:
        f.write(f"{x}\n")
with open("./results/patients_18+_morethan_1d_morethan_2_lactate_measurements_infirst_2d.txt", "w") as f:
    for x in patients_more_24h_2lactate_in_first_48h:
        f.write(f"{x}\n")

In [40]:
patientids = list()
with open("./results/patients_18+_morethan_1d_morethan_2_lactate_measurements_infirst_2d.txt", "r") as f:
    patientids.extend(int(x) for x in f.readlines())
patientids = patientids[:5]
patientids

[17786, 16503, 22946, 9465, 6586]

In [41]:
df_parts = pd.read_csv("./datasets/raw_stage/observation_tables/observation_tables_index.csv")
df_parts = df_parts[df_parts["patientid"].isin(patientids)]
patients_sample = list()
for part in df_parts["part"].unique():
    print(part)
    df_part_X = pd.read_parquet(f"./datasets/raw_stage/observation_tables/parquet/part-{part}.parquet")
    for patient in df_parts[df_parts["part"] == part]["patientid"]:
        patients_sample.append(df_part_X[df_part_X["patientid"] == patient].copy())


df_sample = pd.DataFrame()
for df in patients_sample:
    df_sample = df_sample.append(df)
df_sample.sort_values(by=["patientid", "datetime", "entertime"], inplace=True)
df_patient_data = pd.read_csv("./datasets/reference_data/general_table.csv")
df_sample = df_sample.merge(df_patient_data, on="patientid")
df_variables = pd.read_csv("./datasets/reference_data/hirid_variable_reference.csv")
#extract only observation variables
df_variables = df_variables[df_variables["Source Table"] == "Observation"]
df_sample = df_sample.merge(df_variables, left_on="variableid", right_on="ID")
df_sample.drop(labels=["Source Table", "ID"], axis=1, inplace=True)
df_sample.sort_values(by=["patientid", "admissiontime", "datetime", "entertime"], inplace=True)
df_sample.reset_index(drop=True, inplace=True)
df_sample.to_csv("./results/sample.csv")
display(df_sample)

0


,datetime,entertime,patientid,status,stringvalue,type,value,variableid,admissiontime,sex,age,discharge_status,Variable Name,Unit,Additional information
0,2133-07-29 23:05:00.000,2133-07-29 23:28:41.100,6586,8,None,,160.000000,10000450,2133-07-29 23:05:00,F,80,alive,Body height measure,cm,NaN
5,2133-07-29 23:05:00.000,2133-07-29 23:28:41.100,6586,8,None,,60.000000,10000400,2133-07-29 23:05:00,F,80,alive,Body weight,kg,NaN
10,2133-07-29 23:10:38.810,2133-07-29 23:10:37.066,6586,4,,,57.000000,200,2133-07-29 23:05:00,F,80,alive,Heart rate,/min,NaN
18983,2133-07-29 23:10:38.810,2133-07-29 23:10:37.096,6586,4,,,0.300000,210,2133-07-29 23:05:00,F,80,alive,ST elevation,mm,NaN
37330,2133-07-29 23:10:38.810,2133-07-29 23:10:37.130,6586,4,,,0.000000,212,2133-07-29 23:05:00,F,80,alive,ST elevation,mm,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91362,2162-08-05 13:26:54.110,2162-08-05 13:27:23.540,22946,8,,,0.501662,30010009,2162-08-03 21:20:00,M,30,alive,End tidal carbon dioxide concentration,mmHg,NaN
91363,2162-08-05 13:29:07.040,2162-08-05 13:31:17.200,22946,8,,,0.501662,30010009,2162-08-03 21:20:00,M,30,alive,End tidal carbon dioxide concentration,mmHg,NaN
91364,2162-08-05 13:31:20.000,2162-08-05 13:36:16.680,22946,8,,,0.573328,30010009,2162-08-03 21:20:00,M,30,alive,End tidal carbon dioxide concentration,mmHg,NaN
91365,2162-08-05 13:32:26.460,2162-08-05 13:33:22.820,22946,8,,,0.573328,30010009,2162-08-03 21:20:00,M,30,alive,End tidal carbon dioxide concentration,mmHg,NaN


In [5]:
df_sample = pd.read_csv("./results/sample.csv", index_col=0)
df_sample.reset_index(drop=True, inplace=True)
display(df_sample)

d:\Python_venvs\P37_pycaret_venv\lib\site-packages\IPython\core\interactiveshell.py:3170: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,datetime,entertime,patientid,status,stringvalue,type,value,variableid,admissiontime,sex,age,discharge_status,Variable Name,Unit,Additional information
0,2133-07-29 23:05:00.000,2133-07-29 23:28:41.100,6586,8,NaN,NaN,160.000000,10000450,2133-07-29 23:05:00,F,80,alive,Body height measure,cm,NaN
1,2133-07-29 23:05:00.000,2133-07-29 23:28:41.100,6586,8,NaN,NaN,60.000000,10000400,2133-07-29 23:05:00,F,80,alive,Body weight,kg,NaN
2,2133-07-29 23:10:38.810,2133-07-29 23:10:37.066,6586,4,NaN,NaN,57.000000,200,2133-07-29 23:05:00,F,80,alive,Heart rate,/min,NaN
3,2133-07-29 23:10:38.810,2133-07-29 23:10:37.096,6586,4,NaN,NaN,0.300000,210,2133-07-29 23:05:00,F,80,alive,ST elevation,mm,NaN
4,2133-07-29 23:10:38.810,2133-07-29 23:10:37.130,6586,4,NaN,NaN,0.000000,212,2133-07-29 23:05:00,F,80,alive,ST elevation,mm,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327394,2162-08-05 13:26:54.110,2162-08-05 13:27:23.540,22946,8,NaN,NaN,0.501662,30010009,2162-08-03 21:20:00,M,30,alive,End tidal carbon dioxide concentration,mmHg,NaN
327395,2162-08-05 13:29:07.040,2162-08-05 13:31:17.200,22946,8,NaN,NaN,0.501662,30010009,2162-08-03 21:20:00,M,30,alive,End tidal carbon dioxide concentration,mmHg,NaN
327396,2162-08-05 13:31:20.000,2162-08-05 13:36:16.680,22946,8,NaN,NaN,0.573328,30010009,2162-08-03 21:20:00,M,30,alive,End tidal carbon dioxide concentration,mmHg,NaN
327397,2162-08-05 13:32:26.460,2162-08-05 13:33:22.820,22946,8,NaN,NaN,0.573328,30010009,2162-08-03 21:20:00,M,30,alive,End tidal carbon dioxide concentration,mmHg,NaN
